In [40]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [41]:
install_requirements()

Installing requirements...
Requirements installed successfully.


In [42]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")
    load_dotenv()

    variables_to_check = ["OPENAI_API_KEY"]

    for var in variables_to_check:
        check_env(var)

In [43]:
setup_env()

OPENAI_API_KEY is set.


In [44]:
from openai import OpenAI
import os


class PromptOptimizer:
    """A simple prompt optimization class that uses OpenAI's Chat API to optimize prompts."""

    def __init__(self, prompt: str):
        """Initializes the PromptOptimizer with the given prompt. """
        self.ai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.prompt = prompt
        self.optimized_prompt = None

    def optimize(self, optimize_instructions = [
        "Refine the given prompt so that it is fit for use in production environments.",
        "Make the prompt concise, detailed and clear with a good balance of effectivenss and usefulness.",
    ], turns = 3) -> str:
        """Optimizes the prompt based on optimize instructions. """

        prompt_to_optimize = self.prompt
        previous_optimized_prompt = prompt_to_optimize
        for i in range(turns):
            try:
                print(f"Optimizing prompt - Turn {i+1}...")
                optimize_prompt = f"""
                    You will be given a prompt, refine it as per the given instructions. 
                    In addition to the instructions, come up with improvements and apply the refinements to the prompt.
                    Respond only with the final refined prompt after all improvements are made.
                    Instructions: ```{"\n".join(optimize_instructions)}```
                    Prompt: {previous_optimized_prompt}
                """
                system = "You are an AI prompt optimizer with deep understanding of LLM technology."
                ai_response = self.ai.chat.completions.create(
                    messages=[
                        {"role": "system", "content": system},
                        { "role": "user", "content": optimize_prompt }
                    ],
                    model="gpt-4o-mini"
                )
                previous_optimized_prompt = ai_response.choices[0].message.content
            except Exception as e:
                print(f"Failed to optimize prompt for turn {i+1}: {e}")
                continue
        self.optimized_prompt = previous_optimized_prompt
        return self.optimized_prompt
    
    def get_optimized_prompt(self) -> str:
        """Returns the optimized prompt."""
        if not self.optimized_prompt:
            optimized_prompt = self.optimize()
            if not optimized_prompt:
                print("Failed to optimize the prompt.")
        return self.optimized_prompt
    

    def evaluate(self):
        try:
            print("Evaluating the optimized prompt...")
            optimized_prompt = self.get_optimized_prompt()

            if not optimized_prompt:
                print("Failed to get the optimized prompt, can't evaluate.")
                return None
            
            evaluation_prompt = f"""
                    Given two prompts, compare the original prompt and the optimized prompt. 
                    Provide a table with the comparison of the two prompts along with reasons and scores.
                    In the markdown response provide a title, summary, and conclusion and the table. 
                    Respond in markdown format strictly. 
                    Make sure the markdown tables are compatible with Jupyter notebooks.
                    Original Prompt: {self.prompt}
                    Optimized Prompt: {optimized_prompt}
                """
            
            response = self.ai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are expert in prompt evaluation."},
                    {"role": "user", "content": evaluation_prompt}
                ])
            
            header = f"""# Prompt Optimization Report\n## Original Prompt\n{self.prompt}\n## Optimized Prompt\n{self.optimized_prompt}"""
            report = response.choices[0].message.content
            full_report = header + report
            return full_report
        except Exception as e:
            print(f"Failed to evaluate the optimized prompt: {e}")
            return None
    

        
        


In [45]:
from IPython.display import clear_output, Markdown

def run_prompt_optimizer(prompt: str) -> None:
    prompt_optimizer = PromptOptimizer(prompt)
    optimized_prompt = prompt_optimizer.get_optimized_prompt()
    clear_output()
    report = prompt_optimizer.evaluate()
    return Markdown(report)



In [46]:
# Example 1
prompt = "Give me a detailed plan for learning Python Programming in 2025."

run_prompt_optimizer(prompt)


Evaluating the optimized prompt...


# Prompt Optimization Report
## Original Prompt
Give me a detailed plan for learning Python Programming in 2025.
## Optimized Prompt
Develop a comprehensive learning plan for mastering Python programming in 2025. The plan should include:

1. **Key Topics**: Identify essential concepts, such as data types, control structures, functions, object-oriented programming, and libraries.

2. **Resources**: Suggest specific books, online courses, and tutorials that comprehensively cover each topic.

3. **Timeline**: Outline a phased timeline for each topic, including time allocations and milestones for assessment.

4. **Practical Application**: Recommend strategies for applying learned concepts through projects, offering a range of project ideas from beginner to advanced levels.

The plan must be actionable, well-structured, and designed for learners to efficiently enhance their Python skills.# Prompt Evaluation: Original vs. Optimized

## Summary
This evaluation compares the original prompt, which seeks a detailed learning plan for Python programming in 2025, with an optimized version that elaborates on specific requirements. The optimized prompt is structured and focuses on providing actionable insights, making it more effective for generating a comprehensive learning plan.

## Conclusion
The optimized prompt significantly enhances clarity and direction compared to the original prompt. By specifying key areas to cover, the resources needed, and the structure of the plan, the optimized version provides a more holistic and practical approach for learners. This structured guidance likely results in a more effective and relevant response.

| Feature                    | Original Prompt                                             | Optimized Prompt                                                                                                                                 | Reason for Score            | Score (1-10) |
|----------------------------|------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------|---------------|
| Clarity                    | Vague                                                      | Clearly outlines what the learning plan should include (topics, resources, timeline, etc.)                                                   | More focused guidance       | 8             |
| Specificity                | General request                                            | Specific requirements listed for each part of the learning plan, providing clear expectations                                                 | Higher specificity          | 9             |
| Actionability               | Implicit                                                  | Explicit demand for an actionable and well-structured plan, helping learners efficiently enhance their skills                                | Directly supports learner needs | 9            |
| Structure                  | No defined structure                                       | Organized with numbered points, making it easier to follow and comprehend the plan                                                               | Enhanced logic              | 9             |
| Comprehensive               | Lacks details                                             | Covers essential concepts, resources, timeline, and practical applications                                                                       | More comprehensive content   | 10            |

In [47]:
# Example 2

prompt = "Give me the SOP of a conflict resolution strategy for a team of 5 members in a Big 4 Firm."


run_prompt_optimizer(prompt)

Evaluating the optimized prompt...


# Prompt Optimization Report
## Original Prompt
Give me the SOP of a conflict resolution strategy for a team of 5 members in a Big 4 Firm.
## Optimized Prompt
Draft a Standard Operating Procedure (SOP) for conflict resolution tailored for a five-member team within a Big 4 firm. The SOP must encompass the following components:

1. **Conflict Identification**: Establish clear criteria for early detection of conflicts.
2. **Resolution Procedures**: Outline detailed, step-by-step methods for addressing and resolving conflicts.
3. **Roles and Responsibilities**: Specify each team member's responsibilities in the conflict resolution process.
4. **Collaboration Strategies**: Provide guidelines to foster a collaborative team environment.

Ensure the SOP emphasizes effective communication, accountability, and best practices for conflict prevention throughout the document.# Prompt Comparison: Original vs. Optimized

## Summary
In this analysis, we compare an original prompt requesting a standard operating procedure (SOP) for conflict resolution in a small team at a Big 4 firm with an optimized version of the same prompt. The optimized prompt not only specifies the requirements more clearly but also introduces structured components that make it easier to understand and execute.

## Conclusion
The optimized prompt demonstrates a clearer structure and provides specific guidelines, making it more effective for generating a comprehensive and practical SOP. While the original prompt is straightforward, the optimized version enhances clarity and completeness, which ultimately increases the likelihood of generating a useful document.

| Aspect                      | Original Prompt                                                                                               | Optimized Prompt                                                                                                           | Reason for Score          | Score (Out of 10) |
|-----------------------------|---------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------|---------------------------|--------------------|
| Clarity                     | General request without specifics.                                                                            | Clearly outlines components to include and specifies the structure of the SOP.                                          | More clear instructions.  | 8                  |
| Specificity                 | Asks for a conflict resolution strategy without detailed guidance.                                            | Details what should be included in the SOP such as roles, strategies, and identification criteria.                       | Provides guidance for completeness. | 9                  |
| Structure                   | Lacks any organizational framework.                                                                            | Provides an organized structure with numbered components, facilitating ease of writing and understanding.                | Easier to follow and execute. | 9                  |
| Components                   | Does not specify any required components.                                                                       | Lists essential components for a comprehensive SOP covering all important aspects of conflict resolution.                | Ensures a holistic view.  | 9                  |
| Emphasis on Best Practices   | No emphasis on communication or prevention strategies.                                                        | Emphasizes effective communication, accountability, and best practices, ensuring proactive conflict management.           | Fosters a healthy environment. | 9                  |
| Overall Effectiveness        | May yield a basic SOP but lacks depth and detail.                                                             | Likely to produce a robust, actionable SOP that addresses various scenarios and needs of the team.                      | Higher likelihood of useful output. | 9                  |

Overall, the optimized prompt scores higher across most categories, making it the superior choice for generating a detailed and functional SOP for conflict resolution in a team context.